# Labelling

To assess our results, we labelled 125 french tweets and looked at if the algorithms were able to give the same label as we did. 
To label the tweets, we first decided on the different labels based on the french government policies, the subject we saw during the exploratory data analysis and the actuality in french society. Then, we used crowdsourcing methods where we three labelled independently the tweets and we then aggregated the results using majority vote.

To compare the labelling with the results of the algorithms, we assign to the topics of the algorithms the same labels that we use for the labelling process. Finally, we compare the label of the 125 tweets to the label of the topic it has been assigned by the algorithm and get take the accuracy. (Peut-être que c'est mieux de prendre le F1 score mais peut-être qu'on s'en fout) 

In [5]:
import pandas as pd
import sys
sys.path.append('../helpers_python')
from helpers import *

In [7]:
df = pd.read_csv('../data/to_be_clustered.csv.gz', compression="gzip")
country = 'France'

In [15]:
df.columns

Index(['Unnamed: 0', 'created_at', 'text', 'source', 'translated', 'clean',
       'whcs', 'duplicated', 'language'],
      dtype='object')

In [8]:
categories = {}
categories['France'] = ['white/black - racism', 'islam', 'terrorism', 'coronavirus - vaccination', 
          'american elections', 'India - Pakistan', 'jew - antisemtism', 'farright', 'anti communism'
          'tweeter acccounts', 'fantasy sex play', 'Alain Soral', 'immigration', 'homophobia', 'other']

In [4]:
def prepare_xlsx_for_labelling(country, nb, text_column = 'translated'):
    df_c =pd.DataFrame(df[df.whcs == country][text_column])
    assert nb < len(df_c)
    df_c = df_c.sample(nb)
    for cat in categories[country]:
        df_c[cat] = 0
    df_c.to_excel('data/labelling/'+country+"_to_be_labeled.xlsx")

In [16]:
prepare_xlsx_for_labelling('France', 200, text_column = ['text', 'translated', 'source'])

## Get back the labeled tweets

In [14]:
df_lab = pd.read_excel('../data/labelling/'+country+"_labeled.xlsx")
categories = ['white/black - racism',
       'islam', 'terrorism', 'coronavirus - vaccination', 'american elections',
       'India - Pakistan', 'jew - antisemtism', 'farright',
       'communism - against bankers , against state  prder ',
       'fantasy sex play', 'Alain Soral', 'immigration', 'homophobia',
       'tweeter acccounts', 'women', 'other', "Don't know"]
df_lab[categories] = df_lab[categories].astype(int)
df_lab.rename(columns={'Unnamed: 0' : 'index'}, inplace = True)
df_lab.set_index('index', inplace = True)

In [16]:
df_lab['sum'] = df_lab[categories].sum(axis = 1)
df_lab = df_lab[df_lab['sum'] > 0]
df_lab = df_lab[df_lab["Don't know"] != 1]
df_lab['labels'] = df_lab[categories].apply(lambda row: [col for col, b in zip(categories, row) if b],axis=1)
df_out = df_lab[['translated','labels']]

In [55]:
df_out.to_csv('data/labelling/'+country+'_final.csv.gz', compression="gzip")